In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .config("spark.driver.extraJavaOptions", "-Dlog4j.configuration=file:log4j.properties") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

2026-02-09T12:41:04.393775845Z main ERROR Reconfiguration failed: No configuration found for '4e0e2f2a' at 'null' in 'null'
2026-02-09T12:41:05.049738230Z main ERROR Reconfiguration failed: No configuration found for 'Default' at 'null' in 'null'


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/09 12:41:05 WARN Utils: Your hostname, codespaces-8b0c36, resolves to a loopback address: 127.0.0.1; using 10.0.0.107 instead (on interface eth0)
26/02/09 12:41:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/09 12:41:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, total_amount=33.45),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, total_amount=12.74)]

In [5]:
from datetime import datetime

start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [8]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [9]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [10]:
from collections import namedtuple

In [11]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [12]:
from pyspark.sql import types

In [15]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [16]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [17]:
df_result.show()

[Stage 3:=======================================>                   (2 + 1) / 3]

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-23 13:00:00|  74|1044.0499999999993|   60|
|2020-01-15 11:00:00| 179|              50.5|    5|
|2020-01-16 08:00:00|  41| 736.1399999999996|   54|
|2020-01-31 11:00:00| 260|            126.32|    7|
|2020-01-02 08:00:00|  66|            197.69|   10|
|2020-01-16 14:00:00|  74| 895.1799999999994|   60|
|2020-01-23 12:00:00|  37|             43.35|    2|
|2020-01-10 22:00:00|  95| 407.7100000000002|   37|
|2020-01-13 19:00:00| 223|213.83000000000004|   19|
|2020-01-21 18:00:00| 166| 685.3899999999999|   50|
|2020-01-23 15:00:00| 166| 901.6799999999995|   59|
|2020-01-07 18:00:00|  25| 554.2900000000001|   37|
|2020-01-01 01:00:00| 181| 730.1499999999997|   25|
|2020-01-18 07:00:00|  55|              48.3|    1|
|2020-01-20 00:00:00|  74|221.33000000000004|   16|
|2020-01-11 16:00:00|  65|            936.16|   43|
|2020-01-03 

Traceback (most recent call last):                                              
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 233, in manager
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 87, in worker
BrokenPipeError: [Errno 32] Broken pipe


In [18]:
df_result.write.parquet('tmp/green-revenue')

In [13]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [14]:
import pandas as pd

In [15]:
rows = duration_rdd.take(10)

In [28]:
df = pd.DataFrame(rows, columns=columns).astype({"lpep_pickup_datetime" : str})

VendorID                float64
lpep_pickup_datetime        str
PULocationID              int64
DOLocationID              int64
trip_distance           float64
dtype: object

In [30]:
def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [37]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    df['VendorID'] = df['VendorID'].fillna(-1)
    df = df.astype({"VendorID": int, "lpep_pickup_datetime" : str})
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [38]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [39]:
df_predicts.show()

[Stage 12:>                                                         (0 + 1) / 1]

+--------+--------------------+------------+------------+-------------+------------------+
|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|predicted_duration|
+--------+--------------------+------------+------------+-------------+------------------+
|       2| 2020-01-23 13:10:15|          74|         130|        12.77|63.849999999999994|
|      -1| 2020-01-20 15:09:00|          67|          39|          8.0|              40.0|
|       2| 2020-01-15 20:23:41|         260|         157|         1.27|              6.35|
|       2| 2020-01-05 16:32:26|          82|          83|         1.25|              6.25|
|       2| 2020-01-29 19:22:42|         166|          42|         1.84| 9.200000000000001|
|       2| 2020-01-15 11:07:42|         179|         223|         0.76|               3.8|
|       2| 2020-01-16 08:22:29|          41|         237|         3.32|16.599999999999998|
|       2| 2020-01-28 17:05:28|          75|         161|         2.21|             11.05|

Traceback (most recent call last):                                              
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 233, in manager
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 87, in worker
BrokenPipeError: [Errno 32] Broken pipe
